<a href="https://colab.research.google.com/github/KOCORA/Deep-Learning-.-.-/blob/master/09_Character_Level_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар 9: Character-Level LSTM

## Вступление
На прошлом занятии мы познакомились с тем, как можно векторизовать текстовые данные для решения задач обработки текстов. Сегодня мы продолжим заниматься текстами и посмотрим на простейший пример автоматической генерации текстов при помощи Recurrent Neural Network (RNN).

Полезные материалы по RNN можно почитать [здесь](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), а реализацию на PyTorch — [здесь](https://github.com/karpathy/char-rnn).

### План семинара
1. Подготовка данных
2. Имплементация модели
3. Обучение модели
4. Применение модели

In [1]:
!pip install lightning -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.6/801.6 kB 16.8 MB/s eta 0:00:00


In [2]:
import warnings
from typing import Iterable, Tuple

import pytorch_lightning as pl
import torch
from torch import nn

warnings.filterwarnings("ignore")

## 1. Подготовка данных

### Загрузим текст "Анны Карениной"

In [3]:
!wget https://raw.githubusercontent.com/hse-ds/iad-deep-learning/master/2022/seminars/sem09/anna.txt

--2024-03-06 21:11:57--  https://raw.githubusercontent.com/hse-ds/iad-deep-learning/master/2022/seminars/sem09/anna.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1985223 (1.9M) [text/plain]
Saving to: ‘anna.txt’

anna.txt            100%[===================>]   1.89M  9.56MB/s    in 0.2s    

2024-03-06 21:11:57 (9.56 MB/s) - ‘anna.txt’ saved [1985223/1985223]



In [4]:
with open("anna.txt", "r") as f:
    text = f.read()

text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

### Токенизируем текст

Аналогично предыдущему семинару, в ячейках ниже создадим два словаря для преобразования символов в целые числа и обратно.

In [5]:
unique_chars = tuple(set(text))
int2char = dict(enumerate(unique_chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = torch.tensor([char2int[ch] for ch in text])
encoded[:100]

tensor([19,  5, 50, 75, 66, 53, 11, 17, 67, 38, 38, 38, 24, 50, 75, 75, 48, 17,
         8, 50, 62, 20, 55, 20, 53, 46, 17, 50, 11, 53, 17, 50, 55, 55, 17, 50,
        55, 20, 32, 53, 33, 17, 53, 68, 53, 11, 48, 17, 59,  9,  5, 50, 75, 75,
        48, 17,  8, 50, 62, 20, 55, 48, 17, 20, 46, 17, 59,  9,  5, 50, 75, 75,
        48, 17, 20,  9, 17, 20, 66, 46, 17,  3, 44,  9, 38, 44, 50, 48,  2, 38,
        38, 26, 68, 53, 11, 48, 66,  5, 20,  9])

Посмотрим на схему char-RNN:
<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="30%">

Сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный маппинг), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте напишем функцию для этого преобразования.

#### Задание: допишите функцию one-hot кодирования последовательности

In [8]:
def one_hot_encode(int_words: torch.Tensor, n_labels: int) -> torch.Tensor:
    """
    Creates one-hot representation matrix for a given batch of integer sequences
    :param int_words: tensor of ints, which represents current sequence; shape: [batch_size, seq_len]
    :param n_labels: vocabulary size (number of unique tokens in data)
    :return: one-hot representation of the input tensor; shape: [batch_size, seq_len, n_labels]
    """
    # <YOUR CODE HERE>
    words_one_hot = torch.zeros((int_words.numel(), n_labels), dtype=torch.float32, device=int_words.device)
    words_one_hot[torch.arange(words_one_hot.shape[0]), int_words.flatten()] = 1.0
    words_one_hot = words_one_hot.reshape((*int_words.shape, n_labels))

    return words_one_hot

In [9]:
# testing the function
test_seq = torch.tensor([[3, 5, 1], [0, 2, 4]])
test_one_hot = one_hot_encode(test_seq, 8)

print(test_one_hot)

tensor([[[0., 0., 0., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 1., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0.]]])


### Сформируем батчи
На простом примере батчи будут выглядеть так: мы возьмем закодированные символы и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>

**1. Отбросим часть текста, чтобы у нас были только полные батчи**

Каждый батч содержит $N \times M$ символов, где $N$ — это количество последовательностей в батче (`batch_size`), а $M$ — длина каждой последовательности (`seq_length`). Затем, чтобы получить общее количество батчей $K$, которое мы можем сделать из последовательности, нужно разделить длину последовательности на количество символов в батче. Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из последовательности: $N \times M \times K$.

**2. Разделим текст на $N$ частей**

Этот шаг нужен, чтобы мы могли проходить по тексту окном размера `[batch_size, seq_len]`. Его можно реализовать при помощи простого `reshape`.

**3. Теперь, когда у нас готова матрица текста, мы можем двигаться по ней окном, чтобы получить батчи**

Из каждой позиции окна сформируем обучающие пары `(x, y)` следующим образом: $x$ — это все элементы окна кроме последнего столбца, а $y$ — это все элементы окна кроме первого столбца. Тем самым для каждого токена исходного текста мы будем предсказывать следующий за ним токен.

#### Задание: допишите функцию генерации батчей

In [11]:
def get_batches(
    int_words: torch.Tensor, batch_size: int, seq_length: int
) -> Iterable[torch.Tensor]:
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    # YOUR CODE HERE
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    # YOUR CODE HERE
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        # YOUR CODE HERE
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

In [12]:
# testing the function
test_batches = get_batches(encoded, 8, 50)
test_x, test_y = next(test_batches)
assert test_x.shape == test_y.shape
print(f"x:\n{test_x[:10, :10]}\n")
print(f"y:\n{test_y[:10, :10]}")

x:
tensor([[19,  5, 50, 75, 66, 53, 11, 17, 67, 38],
        [75, 75, 53, 41, 17, 50,  9, 41, 17, 46],
        [66,  5, 50, 66, 17, 20,  9, 46,  3, 55],
        [53, 44, 17, 66,  5,  3, 59, 40,  5, 66],
        [53, 11, 82, 17,  5, 53, 17,  5, 50, 41],
        [11, 82, 17, 44,  5,  3, 17, 44, 50, 46],
        [46, 66, 17, 16, 53, 17, 76,  3, 68, 53],
        [82, 17, 49, 55, 53, 34, 53, 48, 17, 49]])

y:
tensor([[ 5, 50, 75, 66, 53, 11, 17, 67, 38, 38],
        [75, 53, 41, 17, 50,  9, 41, 17, 46,  5],
        [ 5, 50, 66, 17, 20,  9, 46,  3, 55, 59],
        [44, 17, 66,  5,  3, 59, 40,  5, 66,  8],
        [11, 82, 17,  5, 53, 17,  5, 50, 41, 17],
        [82, 17, 44,  5,  3, 17, 44, 50, 46, 17],
        [66, 17, 16, 53, 17, 76,  3, 68, 53, 11],
        [17, 49, 55, 53, 34, 53, 48, 17, 49, 55]])


### Наконец, подготовим класс датасета

In [13]:
class AnnaData(torch.utils.data.IterableDataset):
    def __init__(self, int_words: torch.Tensor, batch_size: int, seq_length: int):
        self.int_words = int_words
        self.batch_size = batch_size
        self.seq_length = seq_length

    def __iter__(self):
        return get_batches(self.int_words, self.batch_size, self.seq_length)

## 2. Имплементация модели

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=50%>

### Структура модели

* Создаём и храним необходимые словари.
* Определяем слой [LSTM]((https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM)) с помощью инстанса класса `torch.nn.LSTM`, который принимает набор параметров: `input_size` — длина последовательности в батче; `n_hidden` — размер скрытых слоёв; `n_layers` — количество слоёв; `drop_prob` — вероятность дропаута; и `batch_first` — флаг, указывающий на то, что у входных последовательностей размерность батча идёт вдоль нулевой оси.
* Определяем слой Dropout с таким же значением `drop_prob`.
* Определяем полносвязный слой с набором параметров: размерность ввода — `n_hidden`; размерность выхода — размер словаря.
* Наконец, инициализируем веса и начальное скрытое состояние (`self.init_hidden()`).

In [14]:
class CharRNN(nn.Module):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 256,
        n_layers: int = 2,
        drop_prob: float = 0.5,
    ) -> None:
        super().__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.drop_prob = drop_prob

        # create mappings
        self.unique_tokens = unique_tokens
        self.int2char = dict(enumerate(self.unique_tokens))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## define the LSTM, dropout and fully connected layers
        self.lstm = nn.LSTM(
            len(self.unique_tokens),
            n_hidden,
            n_layers,
            dropout=drop_prob,
            batch_first=True,
        )
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, len(self.unique_tokens))

    def forward(
        self, x: torch.Tensor, hidden: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view. You may need to use contiguous to reshape the output.
        out = out.contiguous().view(-1, self.n_hidden)
        ## Get the output for classification.
        out = self.fc(out)
        return out, hidden

    def init_hidden(
        self, batch_size: int, weight_device: torch.device
    ) -> Tuple[torch.Tensor]:
        """
        Creates two new zero tensors for hidden state and cell state of LSTM
        :param batch_size: number of sequences per batch
        :param weight_device: torch.device("cuda") for GPU init or torch.device("cpu") for CPU init
        :return: tuple of two tensors of shape [n_layers x batch_size x n_hidden]
        """
        weight = next(self.parameters()).data
        hidden = (
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
            weight.new(self.n_layers, batch_size, self.n_hidden)
            .zero_()
            .to(weight_device),
        )
        return hidden

In [15]:
class CharRNNModule(pl.LightningModule):
    def __init__(
        self,
        unique_tokens: Tuple[str],
        n_hidden: int = 1024,
        n_layers: int = 2,
        drop_prob: float = 0.5,
        batch_size: int = 128,
        seq_length=256,
        lr: float = 0.001,
    ) -> None:
        super().__init__()
        self.model = CharRNN(unique_tokens, n_hidden, n_layers, drop_prob)
        self.hidden = None
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)
        self.n_chars = len(unique_tokens)
        self.batch_size = batch_size
        self.seq_length = seq_length

    def training_step(
        self, train_batch: Tuple[torch.Tensor, torch.Tensor]
    ) -> torch.Tensor:
        x, y = train_batch
        x, y = x.squeeze(0), y.squeeze(0)
        x = one_hot_encode(x, self.n_chars)

        if self.hidden is None:
            self.hidden = self.model.init_hidden(self.batch_size, self.device)
        self.hidden = tuple([each.data for each in self.hidden])

        output, self.hidden = self.model(x, self.hidden)
        loss = self.loss(output, y.reshape(self.batch_size * self.seq_length).long())

        self.log("train_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer

## 3. Обучение модели

По классике, используем оптимизатор Adam и кросс-энтропию. Но без пары особенностей не обойтись:
* Во время цикла будем отделять скрытое состояние от его истории, потому что скрытое состояние LSTM является кортежем скрытых состояний.
* Будем использовать gradient clipping, чтобы избавиться от взрывающихся градиентов.

In [18]:
# data
train_dataset = AnnaData(encoded, batch_size=128, seq_length=256)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,  # batching is already implemented on our side
    shuffle=False,
    num_workers=1,  # don't change: it will lead to the wrong implementation
)
# model
char_rnn = CharRNNModule(unique_chars, n_hidden=1024, batch_size=128)
# trainer
trainer = pl.Trainer(max_epochs=1, gradient_clip_val=1.0, accelerator="cpu", devices=1)
trainer.fit(char_rnn, train_dataloaders=train_dataloader)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type             | Params
-------------------------------------------
0 | model | CharRNN          | 13.0 M
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
13.0 M    Trainable params
0         Non-trainable params
13.0 M    Total params
52.097    Total estimated model params size (MB)


Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


## 4. Применение модели

Сперва сохраним обученную модель, чтобы можно было загрузить её позже. В следующей ячейке сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [25]:
net = char_rnn.model
checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.unique_tokens,
}

with open("rnn_x_epoch.net", "wb") as f:
    torch.save(checkpoint, f)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Делаем предсказания

Сгенерируем текст! Для предсказания продолжения текста мы передаём в сеть последний символ, она предсказывает следующий символ, который мы снова передаем на вход, получаем ещё один предсказанный символ и так далее. Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов на каждом шаге генерации, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые, наиболее вероятные символы. С одной стороны, такой подход позволит нам рассматривать не только самую вероятную последовательность с точки зрения прогноза модели. С другой стороны, мы будем работать с ограниченным набором сгенерированных вариантов, поэтому избавимся от совсем уж шумовых прогнозов. Узнать больше можно [здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [26]:
def predict_next_char(
    model: torch.nn.Module, char: str, h: torch.Tensor = None, top_k: int = None
) -> Tuple[str, torch.Tensor]:
    """
    Given a character and a model, predicts next character in the sequence
    :param model: model that outputs next token probability distribution
    :param char: last character of the sequence to continue generation from
    :param h: hidden state of the model
    :param top_k: number of most probable tokens to be chosen from
    :return: tuple of next character and new hidden state
    """
    # tensor inputs
    x = torch.tensor([[model.char2int[char]]])
    x = one_hot_encode(x, len(model.unique_tokens))
    x = x.to(device)

    h = tuple([each.data for each in h])

    out, h = model(x, h)

    # get the character probabilities
    p = torch.nn.functional.softmax(out, dim=1).data.cpu()

    # get top characters
    if top_k is None:
        top_ch = torch.arange(len(model.unique_tokens))
    else:
        p, top_ch = p.topk(top_k)

    p.squeeze_()
    top_ch.squeeze_()
    char = top_ch[torch.multinomial(p / p.sum(), 1)]

    return model.int2char[char.item()], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [27]:
def sample(model, size, prime="The", top_k=None):
    model.to(device)
    model.eval()

    # run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1, device)
    for ch in prime:
        char, h = predict_next_char(model, ch, h, top_k=top_k)

    chars.append(char)

    # pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict_next_char(model, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [22]:
print(sample(net, 1000, prime="Anna", top_k=5))

Anna e e a eeeoeae oa ee ae  e   ee   oaoet eato  ttoot ea oetteteee eeoo e  e ea  eoot  teeoe o eao   oa e  oae  e  ttoo ea ett   ee  atea o t  taottea eeeoeo  e tteo t tooo  t eee  a tee t eo  eet  attaoateo  e o tt teoo  ott ea otao te a o t   tot  e   e ee e oaae   oeoooea  e e aoe  oteeaot eto   atoeoea teoeet t ottae  t ooaoot eeoot eo e  te    e tee  ee e ae    e oa t  tet e  to     o t  t aoe a ete    eee ttea aoatt t taee to eoeet taeae  aeeto o   e eat eotoe tto aeaat teoataa e  eatat atta ea o o eo e et aoote    at ea   a taete tt  ea tttoeetttae  o    eto e eoe  et ae t o eea  eate oeaatat t  t a atoato   o to oee  oeea eoae   oa   teeeataea t e  aeaatae eeea ooee otttootoe  teee tte  tteat a ee a teaea at  ee  eoeoeot oeateaaoeaaaaoaoet aeo  toteo  oao  ea e oto tet    to  e  aaae  a aeeaoo  at oat aa  eteo  teateo ett  aaaet ttt eaa e to eoa    e eoe ttaaet aoeta     tao oeo o  eto o   teoaee  eoete  totottaa e etee e et   e e   e oatetotaea ooeooatt teoe aaaet   oea tet 

### Загрузка чекпоинта и генерация

In [28]:
with open("rnn_x_epoch.net", "rb") as f:
    checkpoint = torch.load(f)

loaded = CharRNN(
    checkpoint["tokens"],
    n_hidden=checkpoint["n_hidden"],
    n_layers=checkpoint["n_layers"],
)
loaded.load_state_dict(checkpoint["state_dict"])

# sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))

And Levin saideet  e   etto  ttae aao aaoee oe  ett e   ta a  o  a t   o    ettaett eaa eea a et    e ttoea o a   eeato t   a a ota  e    aae oo o eooee eo oe aat  toe   e ooa ooa t ete e eaoeee te to  tto   tet  o  aoe e et      aoa   aao oo   to e aaeaooattaae  etteeeeeoa ooe a  ea  a o      tte e  eeaa eoteoa o ta  e  e teteaeattaoee   ta oeate t ot ot ae oeoteo   eo t  a tee  e t a     atteoe oae o e   at t oeo    ot t teto tae t oeoe  ot ta o  e o oaeaeeea  a a otaee ttoee ete t oa te eeoe  o oea eota t tt tto ae e aettooee  o e   eo  oa aa  o  eet e   ttaea    a toeo   et o  a  attte et eaeaa e ato  ete  e t e   tetoaoot oeee e eotee e ea a o  to e    oeo  t o otea a    tt at aeoea oat  o  eo e t teee aeooaeeoot   o e aeatoa   aeo ttttae ooaoo o  aaea  teate taaatoet e oa tatoe tottaea oae  oe e ao e e t oe ee  o e aaeaaa eee   e te t toeee  o aeat  e aa   to  o ae   a t  a ae   e t aa  eoaooo aet  e oo ot eea  o a e atotetaoe  o t  t a  ttaae o e o oeta eo o teo eot   a oaota tt